In [ ]:
!pip install openai==0.28

In [ ]:
!pip install langchain

In [ ]:
!pip install langchain_community

In [4]:
!pip install pymupdf==1.22.5

In [ ]:
!pip install fitz

In [ ]:
!pip install tiktoken

In [ ]:
!pip install chromadb

In [ ]:
!python -m spacy download ko_core_news_sm

## Load documents

In [ ]:
# langchain 라이브러리에서 필요한 모듈들을 가져옵니다.
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from collections import Counter
from langchain.document_loaders import PyMuPDFLoader  # PDF 파일을 읽어오는 데 사용
from langchain.embeddings import OpenAIEmbeddings  # OpenAI의 임베딩 기능을 사용
from langchain.text_splitter import SpacyTextSplitter  # 텍스트를 적절한 크기로 나누기
from langchain.vectorstores import Chroma  # 벡터 데이터를 저장하고 검색하는 데 사용

# PDF 파일을 읽기 위한 로더를 초기화합니다.
loader = PyMuPDFLoader("/content/test_sample.pdf")
documents = loader.load()  # 파일을 읽어 문서 데이터를 가져옵니다.

# 문서를 처리하기 좋은 크기의 조각으로 나누는 스플리터를 설정합니다.
# 'ko_core_news_sm'은 한국어 처리를 위한 Spacy 모델입니다.
text_splitter = SpacyTextSplitter(
    chunk_size=300,
    pipeline="ko_core_news_sm"
)
splitted_documents = text_splitter.split_documents(documents)  # 문서를 나눕니다.

# OpenAI의 임베딩 모델을 초기화합니다. 여기서는 'text-embedding-ada-002' 모델을 사용합니다.
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=""
)

In [10]:
vectordb = Chroma(
    persist_directory="./.data",  # 데이터 저장 경로 지정
    embedding_function=embeddings  # 임베딩 함수 지정
)

## Make a retriever

In [11]:
retriever = vectordb.as_retriever()


In [12]:
documents = retriever.get_relevant_documents("현충일 제정일은?")
print(f"문서 수: {len(documents)}") # 문서 개수 표시

for document in documents:
    print(f"문서 내용: {document.page_content}") # 문서 내용을 표시


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


문서 수: 0


### 결과를 k개 반환

In [13]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [14]:
documents = retriever.get_relevant_documents("현충일에 대해서 설명해줘!?")

for document in documents:
    print(f"문서 내용: {document.page_content}")

## Make a chain

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0,
                   openai_api_key=""),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

## Query

In [ ]:
input_text = "문재인 대통령의 취임일은?"
chatbot_response = qa_chain(input_text)
print(chatbot_response)

In [ ]:
!pip install gradio

To create a public link, set `share=True` in `launch()`.  
Running on https://localhost:7860/  

라는 식의 문구가 나오면 위의 localhost로 시작하는 주소를 클릭하세요.

In [ ]:
import gradio as gr

# 인터페이스를 생성하고 관리하는 블록 구조를 사용
with gr.Blocks() as demo:
    # Chatbot: 그라디오의 챗봇 컴포넌트를 생성하여 인터페이스에 추가
    chatbot = gr.Chatbot(label="PDF 검색챗봇")  # 챗봇 상단에 "PDDF검색챗봇" 레이블이 표시됨
    # Textbox: 사용자의 질문을 입력할 수 있는 텍스트 상자를 생성하여 추가
    msg = gr.Textbox(label="질문 해주세요!")  # 하단의 채팅창에 "질문 해주세요!" 레이블이 표시됨
    # Button: 대화를 초기화할 수 있는 버튼을 생성
    clear = gr.Button("대화를 초기화합니다")  # "대화를 초기화합니다" 레이블의 버튼이 표시됨

    # respond 함수: 사용자의 메시지를 받아 챗봇의 응답을 반환하는 함수
    def respond(message, chat_history):
        # qa_chain 함수: 입력된 질문을 기반으로 답변을 생성 (이 부분은 실제 구현에 따라 다름)
        result = qa_chain(message)
        bot_message = result['result']  # 챗봇의 답변을 가져옴
        bot_message += ' # sources :'  # 답변 출처를 추가하기 위한 태그

        # 각 출처 문서를 참조하여 답변에 출처 정보 추가
        for i, doc in enumerate(result['source_documents']):
            bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + ' '

        # 사용자의 질문과 챗봇의 답변을 채팅 기록에 추가
        chat_history.append((message, bot_message))
        # 새 메시지를 입력할 수 있도록 입력 상자를 비우고, 업데이트된 채팅 기록 반환
        return "", chat_history

    # 사용자가 텍스트 상자에 입력한 메시지를 제출하면 respond 함수가 호출
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행: 웹 인터페이스를 브라우저에서 실행 가능
demo.launch(debug=True)
